In [7]:
import pandas as pd
import string

sector_letters = list(string.ascii_uppercase)[:21]

dfs = []

kwartaal_map = {
    'Eerste kwartaal': 1, 
    'Tweede kwartaal': 2, 
    'Derde kwartaal': 3, 
    'Vierde kwartaal': 4
}

for letter in sector_letters:
    filename = f"Aantal bedrijven naar omvang {letter}.csv"

    df = pd.read_csv(f"Enterprise_size/{filename}", sep=';')

    cols_to_keep = [
        '1 werkzaam persoon',
        '2 tot 10 werkzame personen',
        '10 tot 50 werkzame personen',
        '50 tot 250 werkzame personen'
    ]

    # Obtain latest quarter each sector has available data (2025 Q4)
    df['Q_sort'] = df['Kwartaal'].map(kwartaal_map)
    df = df.sort_values(by=['Jaar', 'Q_sort'])    
    most_recent_row = df.iloc[[-1]].copy()
    
    subset = most_recent_row[cols_to_keep].copy()
    subset['Sector'] = letter
    
    dfs.append(subset)

all_sectors = pd.concat(dfs, ignore_index=True)

all_sectors = all_sectors.rename(columns={
    '1 werkzaam persoon': '1 worker',
    '2 tot 10 werkzame personen': '2-10 workers',
    '10 tot 50 werkzame personen': '10-50 workers',
    '50 tot 250 werkzame personen': '50-250 workers'
})

all_sectors.set_index('Sector', inplace=True)

all_sectors

,1 worker,2-10 workers,10-50 workers,50-250 workers
Sector,,,,
A,40725,35530,1805,185
B,505,65,40,35
C,61990,16425,5810,2010
D,1800,275,75,35
E,1375,375,180,85
F,234820,30475,4600,775
G,201115,80135,11825,2250
H,53475,11940,2655,715
I,44795,29795,6485,480


In [8]:
sector_groups = {
    "Agriculture, forestry and fishing": ["A"],
    "Industry and energy": ["B", "C", "D", "E"],
    "Construction": ["F"],
    "Trade, transport and hospitality": ["G", "H", "I"],
    "Information and communication": ["J"],
    "Financial services": ["K"],
    "Real estate": ["L"],
    "Business services": ["M", "N"],
    "Government and care": ["O", "P", "Q"],
    "Culture, recreation and other": ["R", "S", "T", "U"]
}

new_data = []

for new_name, letters in sector_groups.items():

    summed_values = all_sectors.loc[letters].sum()    
    summed_values.name = new_name
    new_data.append(summed_values)

combined_df = pd.DataFrame(new_data)
combined_df.loc['TOTAL'] = combined_df.sum()
combined_df['TOTAL'] = combined_df.sum(axis=1)
combined_df

,1 worker,2-10 workers,10-50 workers,50-250 workers,TOTAL
"Agriculture, forestry and fishing",40725,35530,1805,185,78245
Industry and energy,65670,17140,6105,2165,91080
Construction,234820,30475,4600,775,270670
"Trade, transport and hospitality",299385,121870,20965,3445,445665
Information and communication,114260,14695,3090,715,132760
Financial services,29220,5390,775,240,35625
Real estate,30600,8085,505,185,39375
Business services,564565,62250,9960,2240,639015
Government and care,338070,32845,6195,1675,378785
"Culture, recreation and other",247710,26635,2670,535,277550
